In [84]:
import pandas as pd 
import numpy as np
from numpy import array
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from ipynb.fs.full.Data_Processing import extract_test_number, get_user_samples
from sklearn import preprocessing
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
torch.set_printoptions(linewidth=120) #Display options for output
torch.set_grad_enabled(True) # Already on by default
torch.manual_seed(0)
from torch_lr_finder import LRFinder
import pickle
import torch.utils.data as data_utils
from collections import namedtuple

In [85]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 8), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*7, 5)
        
    def forward(self, x):
        # Layer 1
        x = x.float()
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.1)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.1)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.1)
        x = self.pooling3(x)

        # FC Layer
        x = x.view(-1, 4*2*7)
        x = self.fc1(x)
        return x

**Evaluate function returns values of different criteria like accuracy, precision etc.**
In case you face memory overflow issues, use batch size to control how many samples get evaluated at one time. Use a batch_size that is a factor of length of samples. This ensures that you won't miss any samples.

In [86]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted , multi_class="ovr"))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted), average='macro'))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted) , average='macro'))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted) , average='macro')
            recall = recall_score(Y, np.round(predicted) , average='macro')
            results.append(2*precision*recall/ (precision+recall))
    return results

**Generate random data**
    
*Data format:*

Datatype - float32 (both X and Y)

X.shape - (#samples, 1, #timepoints, #channels)

Y.shape - (#samples)

In [89]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()### Scale the data

In [88]:
def learning_rate_finder():
    model = EEGNet()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-7, weight_decay=1e-2)
    lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
    lr_finder.range_test(train_loader, end_lr=100, num_iter=100, step_mode="exp")
    lr_finder.plot()

In [90]:
def get_accuracy(train_loader, test_loader, dataset_type):
    correct = 0
    total = 0
    with torch.no_grad():
        if dataset_type == "test":
            loader = test_loader
        elif dataset_type == "train":
            loader = train_loader
        for data in loader:
            inputs = data[0].cuda(0)
            labels = data[1].cuda(0)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = (correct/total * 100)
    return acc

In [93]:
def train_network(X_train, train_loader, verbose=False ):
    net = EEGNet().cuda(0)
    optimizer = optim.Adam(net.parameters(), lr = 0.001)

    preds_list = []
    labels_list = []
    for epoch in range(500):
        total_loss = 0
        total_correct = 0

        for batch in train_loader:
        #Get Batch

            inputs = batch[0].cuda(0)
            labels = batch[1].cuda(0)


            preds = net(inputs) #Pass batch
        #     criterion=nn.BCEWithLogitsLoss()
            loss = F.cross_entropy(preds, labels.long()) #calculate loss

            optimizer.zero_grad()
            loss.backward()  #calculate gradients
            optimizer.step() #update weights


            preds_list.append(preds)
            labels_list.append(labels)
            total_loss += loss.item()
            total_correct += get_num_correct(preds, labels)
            
        if verbose == True:             
    #     Validation accuracy
    #     params = ["acc", "fmeasure"]
    #     print (params)
    #     print ("Train - ", evaluate(net, X_train, y_train_new, params))
            print("epoch: {0}, Accuracy {1}, loss: {2}".format(epoch, total_correct/len(X_train), loss))

### Training and testing

In [96]:
sampled_file = "User_1_sampled_annotated_EEG.pickle"
with open(sampled_file, 'rb') as handle:
    dt = pickle.load(handle)

labels = ["attention", "interest", "effort"]

results = []
for label in labels:
    
    X = dt["inputs"]
    y = dt[label]

    #Convert the categories into labels 
    le = LabelEncoder()
    y =  le.fit_transform(y)

    #Train/test split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

    # scale the data 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

    #Convert to 4D 
    X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1],X_train.shape[2])
    X_test = X_test.reshape(X_test.shape[0],1, X_test.shape[1],X_test.shape[2])

    #Create train and test loader
    train = data_utils.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train).long())
    test = data_utils.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
    train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)
    test_loader = data_utils.DataLoader(test, batch_size=50, shuffle=False)
    
    # train the network
    train_network(X_train, train_loader, verbose=False)
    
    #get results
    Results = namedtuple("Results","label train_acc test_acc user")
    train_acc = get_accuracy(train_loader, test_loader, "train")
    test_acc = get_accuracy(train_loader, test_loader, "test")
    print("{0} results on User 1\ntrain_acc: {1}\tsample_size: {2}\ntest_acc: {3}\tsample_size: {4}\n".format(label, 
                                                                                           train_acc,  len(X_train),
                                                                                            test_acc, len(X_test)))
    results.append(Results(label, train_acc, test_acc,1))

results = pd.DataFrame(results)
results.to_csv("results/user1_results_120sample_120_slide_bandpass_2048.csv", index=False)
results

attention results on User 1
train_acc: 26.373626373626376	sample_size: 546
test_acc: 27.73722627737226	sample_size: 137

interest results on User 1
train_acc: 34.24908424908425	sample_size: 546
test_acc: 33.57664233576642	sample_size: 137

effort results on User 1
train_acc: 23.626373626373624	sample_size: 546
test_acc: 21.16788321167883	sample_size: 137



,label,train_acc,test_acc,user
0,attention,26.373626,27.737226,1
1,interest,34.249084,33.576642,1
2,effort,23.626374,21.167883,1


**Run**